In [1]:
import pandas as pd
import numpy as np

In [2]:
hosp_input = pd.read_csv('./resultado.csv', decimal=".")
hosp_input.drop(columns=['dist_praia'], inplace=True)

In [3]:
hosp_input['checkin'] = hosp_input['checkin'].astype("category")
hosp_input['checkout'] = hosp_input['checkout'].astype("category")
hosp_input['cidade'] = hosp_input["cidade"].astype("category")
hosp_input['hotel'] = hosp_input["hotel"].astype("category")
hosp_input['data_extracao'] =hosp_input["data_extracao"].astype("category")
hosp_input['avaliacao'] = hosp_input['avaliacao'].replace('None', pd.NA)
hosp_input['avaliacao'] = pd.to_numeric(hosp_input['avaliacao'], errors='coerce')
hosp_input['avaliacao'] =hosp_input["avaliacao"].astype("float")
hosp_input['qtnd_avaliacoes'] = hosp_input['qtnd_avaliacoes'].replace('None', pd.NA)
hosp_input['qtnd_avaliacoes'] = pd.to_numeric(hosp_input['qtnd_avaliacoes'], errors='coerce')
hosp_input['qtnd_avaliacoes'] =hosp_input["qtnd_avaliacoes"].astype("float")


In [4]:
hosp_input['hotel'] = hosp_input['hotel'].cat.codes
hosp_input['checkin'] = hosp_input['checkin'].cat.codes
hosp_input['checkout'] = hosp_input['checkout'].cat.codes
hosp_input['cidade'] = hosp_input['cidade'].cat.codes
hosp_input['data_extracao'] = hosp_input['data_extracao'].cat.codes

In [5]:
hosp_input.head(3)

,hotel,cidade,preço,checkin,checkout,dist_centro,avaliacao,qtnd_avaliacoes,data_extracao
0,6125,0,12144,0,3,0.9,8.5,5355.0,0
1,4485,0,6478,0,3,3.1,8.3,3469.0,0
2,4246,0,4067,0,3,3.9,8.1,4658.0,0


In [6]:
hosp_input.dtypes

hotel                int16
cidade                int8
preço                int64
checkin               int8
checkout              int8
dist_centro        float64
avaliacao          float64
qtnd_avaliacoes    float64
data_extracao         int8
dtype: object

In [7]:
from sklearn.preprocessing import MinMaxScaler

# colunas a serem normalizadas
columns_to_normalize = ['preço', 'dist_centro', 'avaliacao', 'qtnd_avaliacoes']

scaler = MinMaxScaler()

# aplicando a normalização
hosp_input[columns_to_normalize] = scaler.fit_transform(hosp_input[columns_to_normalize])

print(hosp_input)

        hotel  cidade     preço  checkin  checkout  dist_centro  avaliacao  \
0        6125       0  0.027797        0         3     0.016807   0.833333   
1        4485       0  0.014528        0         3     0.063025   0.811111   
2        4246       0  0.008882        0         3     0.079832   0.788889   
3         500       0  0.008620        0         3     0.033613   0.755556   
4        5528       0  0.004393        0         3     0.025210   0.833333   
...       ...     ...       ...      ...       ...          ...        ...   
100645   4367       1  0.026823       49        49     0.012605   0.644444   
100646   5598       1  0.011718       49        49     0.039916        NaN   
100647   1048       1  0.032394       49        49     0.052521   0.888889   
100648   4345       1  0.022275       49        49     0.071429   0.666667   
100649   1949       1  0.013905       49        49     0.096639   0.444444   

        qtnd_avaliacoes  data_extracao  
0              0.33772

In [8]:
!pip install fancyimpute

In [11]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (1.3.2)


In [9]:
from fancyimpute import KNN
hosp_knn = KNN(k=3).fit_transform(hosp_input)

Imputing row 1/100650 with 0 missing, elapsed time: 2323.553
Imputing row 101/100650 with 0 missing, elapsed time: 2323.647
Imputing row 201/100650 with 0 missing, elapsed time: 2323.688
Imputing row 301/100650 with 0 missing, elapsed time: 2323.693
Imputing row 401/100650 with 0 missing, elapsed time: 2323.739
Imputing row 501/100650 with 0 missing, elapsed time: 2323.782
Imputing row 601/100650 with 0 missing, elapsed time: 2323.850
Imputing row 701/100650 with 2 missing, elapsed time: 2323.993
Imputing row 801/100650 with 0 missing, elapsed time: 2324.070
Imputing row 901/100650 with 0 missing, elapsed time: 2324.070
Imputing row 1001/100650 with 0 missing, elapsed time: 2324.070
Imputing row 1101/100650 with 0 missing, elapsed time: 2324.071
Imputing row 1201/100650 with 0 missing, elapsed time: 2324.085
Imputing row 1301/100650 with 0 missing, elapsed time: 2324.112
Imputing row 1401/100650 with 0 missing, elapsed time: 2324.166
Imputing row 1501/100650 with 0 missing, elapsed tim

In [10]:
hosp_knn.shape

(100650, 9)

In [12]:
hosp = pd.DataFrame(data=hosp_knn[0:,0:],columns=['hotel', 'cidade', 'preço', 'checkin', 'checkout', 'dist_centro','avaliacao', 'qtnd_avaliacoes', 'data_extracao'])

In [17]:
# revertendo a normalização
hosp[columns_to_normalize] = scaler.inverse_transform(hosp[columns_to_normalize])

In [18]:
hosp.head()

,hotel,cidade,preço,checkin,checkout,dist_centro,avaliacao,qtnd_avaliacoes,data_extracao
0,6125.0,0.0,12144.0,0.0,3.0,0.9,8.5,5355.0,0.0
1,4485.0,0.0,6478.0,0.0,3.0,3.1,8.3,3469.0,0.0
2,4246.0,0.0,4067.0,0.0,3.0,3.9,8.1,4658.0,0.0
3,500.0,0.0,3955.0,0.0,3.0,1.7,7.8,3126.0,0.0
4,5528.0,0.0,2150.0,0.0,3.0,1.3,8.5,144.0,0.0


In [19]:
print(hosp.isnull().sum())

hotel              0
cidade             0
preço              0
checkin            0
checkout           0
dist_centro        0
avaliacao          0
qtnd_avaliacoes    0
data_extracao      0
dtype: int64


In [13]:
hosp_teste = pd.read_csv('./resultado.csv', decimal=".")

In [14]:
hosp['hotel'] = hosp_teste['hotel']
hosp['cidade'] = hosp_teste['cidade']
hosp['checkin'] = hosp_teste['checkin']
hosp['checkout'] = hosp_teste['checkout']
hosp['data_extracao'] = hosp_teste['data_extracao']

In [15]:
hosp.head(3)

,hotel,cidade,preço,checkin,checkout,dist_centro,avaliacao,qtnd_avaliacoes,data_extracao
0,TURIM Boulevard Hotel,Lisboa,12144.0,2024-06-1,2024-06-8,0.9,8.5,5355.0,2024-5-23
1,Luzeiros Suites,Lisboa,6478.0,2024-06-1,2024-06-8,3.1,8.3,3469.0,2024-5-23
2,Lisbon Serviced Apartments - Campos,Lisboa,4067.0,2024-06-1,2024-06-8,3.9,8.1,4658.0,2024-5-23


In [16]:
hosp_teste

,hotel,cidade,preço,checkin,checkout,dist_centro,dist_praia,avaliacao,qtnd_avaliacoes,data_extracao
0,TURIM Boulevard Hotel,Lisboa,12144,2024-06-1,2024-06-8,0.9,None,8.5,5355,2024-5-23
1,Luzeiros Suites,Lisboa,6478,2024-06-1,2024-06-8,3.1,None,8.3,3469,2024-5-23
2,Lisbon Serviced Apartments - Campos,Lisboa,4067,2024-06-1,2024-06-8,3.9,None,8.1,4658,2024-5-23
3,Amazonia Lisboa Hotel,Lisboa,3955,2024-06-1,2024-06-8,1.7,None,7.8,3126,2024-5-23
4,Rua Luciano Cordeiro Bhuiyan Family Host,Lisboa,2150,2024-06-1,2024-06-8,1.3,None,8.5,144,2024-5-23
...,...,...,...,...,...,...,...,...,...,...
100645,Love Nest in Saint-Michel,Paris,11728,2025-06-22,2025-06-29,0.7,None,6.8,9,2024-5-23
100646,SINGLE STUDIO QuartierLatin,Paris,5278,2025-06-22,2025-06-29,2.0,None,None,None,2024-5-23
100647,Bastille - Gare de Lyon - Appartement familial,Paris,14107,2025-06-22,2025-06-29,2.6,None,9.0,2,2024-5-23
100648,Loft in the heart of Paris,Paris,9786,2025-06-22,2025-06-29,3.5,None,7.0,1,2024-5-23


In [20]:
hosp.to_csv('dataset_imputado.csv', index=False)